### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/121_TROCAS_QE_July2020_DENOVO_21/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/121_TROCAS_QE_July2020_DENOVO_21


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks126 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/121_TROCAS_QE_July2020_DENOVO_21/121_TROCAS_QE_July2020_DENOVO_21_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks126))

print(peaks126.columns)

# These columns mess things up- get rid of them

del peaks126['Fraction']
del peaks126['Scan']
del peaks126['Source File']
del peaks126['Tag Length']
del peaks126['PTM']
del peaks126['tag (>=0%)']
del peaks126['mode']
del peaks126['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks126.columns = columns

mean_len = peaks126['length'].mean()
print(mean_len)

# look at the dataframe
peaks126.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 90
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
10.077777777777778


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,98,10,523.2867,2,37.43,63800000.0,1044.5564,2.3
1,LPQVEGTGGN(+.98)VQPSKDLVR,98,19,998.0143,2,56.30,NaN,1994.0432,-14.6
2,VTN(+.98)AQ(+.98)LAEVLAR,96,12,643.8535,2,71.74,1420000.0,1285.6877,3.6
3,VTN(+.98)AQ(+.98)LAEVLAR,96,12,643.8522,2,71.24,NaN,1285.6877,1.6
4,LSSPATLDSR,96,10,523.7785,2,39.74,219000.0,1045.5403,2.1


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks126['A'] = peaks126['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks126['C'] = peaks126['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks126['D'] = peaks126['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks126['E'] = peaks126['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks126['F'] = peaks126['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks126['G'] = peaks126['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks126['H'] = peaks126['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks126 output, there will be no isoleucines (they're lumped in with leucines)
peaks126['I'] = peaks126['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks126['K'] = peaks126['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks126['L'] = peaks126['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks126['M'] = peaks126['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks126['N'] = peaks126['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks126['P'] = peaks126['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks126['Q'] = peaks126['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks126['R'] = peaks126['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks126['S'] = peaks126['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks126['T'] = peaks126['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks126['V'] = peaks126['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks126['W'] = peaks126['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks126['Y'] = peaks126['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks126['c-carb'] = peaks126['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks126['m-oxid'] = peaks126['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks126['n-deam'] = peaks126['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks126['q-deam'] = peaks126['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks126['stripped_peptide'] = peaks126['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks126['stripped_length'] = peaks126['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks126['ptm-total'] = peaks126['c-carb'] + peaks126['m-oxid'] + peaks126['n-deam'] + peaks126['q-deam']

# calculate NAAF numerator for each peptide k
peaks126['NAAF_num.'] = peaks126['Area'] / peaks126['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50.csv")

# check out the results
peaks126.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,98,10,523.2867,2,37.43,63800000.0,1044.5564,2.3,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,6380000.0
1,LPQVEGTGGN(+.98)VQPSKDLVR,98,19,998.0143,2,56.30,NaN,1994.0432,-14.6,0,...,0,0,0,0,1,0,LPQVEGTGGNVQPSKDLVR,19,1,NaN
2,VTN(+.98)AQ(+.98)LAEVLAR,96,12,643.8535,2,71.74,1420000.0,1285.6877,3.6,3,...,0,0,0,0,1,1,VTNLAEVLAR,10,2,142000.0
3,VTN(+.98)AQ(+.98)LAEVLAR,96,12,643.8522,2,71.24,NaN,1285.6877,1.6,3,...,0,0,0,0,1,1,VTNLAEVLAR,10,2,NaN
4,LSSPATLDSR,96,10,523.7785,2,39.74,219000.0,1045.5403,2.1,1,...,0,0,0,0,0,0,LSSPATLDSR,10,0,21900.0


In [5]:
# keep only stripped peptide I/L and NAAF
dn_126 = peaks126[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_126.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/126B_MN_T00_GF_DN50_ILnaafs.csv")

dn_126.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,63800000.0,6380000.0
1,LPQVEGTGGNVQPSKDLVR,NaN,NaN
2,VTNLAEVLAR,1420000.0,142000.0
3,VTNLAEVLAR,NaN,NaN
4,LSSPATLDSR,219000.0,21900.0


In [6]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks126['A'].sum(),
        'C': peaks126['C'].sum(),
        'D': peaks126['D'].sum(),
        'E': peaks126['E'].sum(),
        'F': peaks126['F'].sum(),
        'G': peaks126['G'].sum(),
        'H': peaks126['H'].sum(),
        'I': peaks126['I'].sum(),
        'K': peaks126['K'].sum(),
        'L': peaks126['L'].sum(),
        'M': peaks126['M'].sum(),
        'N': peaks126['N'].sum(),
        'P': peaks126['P'].sum(),
        'Q': peaks126['Q'].sum(),
        'R': peaks126['R'].sum(),
        'S': peaks126['S'].sum(),
        'T': peaks126['T'].sum(),
        'V': peaks126['V'].sum(),
        'W': peaks126['W'].sum(),
        'Y': peaks126['Y'].sum(),
        'c-carb': peaks126['c-carb'].sum(),
        'm-oxid': peaks126['m-oxid'].sum(),
        'n-deam': peaks126['n-deam'].sum(),
        'q-deam': peaks126['q-deam'].sum(),
        'Total area': peaks126['Area'].sum(),
        'Total length': peaks126['stripped_length'].sum()
       }

totalpeaks126 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks126['% C w/ carb'] = totalpeaks126['c-carb'] / totalpeaks126['C'] 

# calculate percentage of M's that are oxidized
totalpeaks126['% M w/ oxid'] = totalpeaks126['m-oxid'] / totalpeaks126['M'] 

# calculate percentage of N's that are deamidated
totalpeaks126['% N w/ deam'] = totalpeaks126['n-deam'] / totalpeaks126['N'] 

# calculate percentage of N's that are deamidated
totalpeaks126['% Q w/ deam'] = totalpeaks126['q-deam'] / totalpeaks126['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks126['NAAF denom.'] = totalpeaks126['Total area'] / totalpeaks126['Total length']

# write modified dataframe to new txt file
totalpeaks126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50_totals.csv")

totalpeaks126.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,80,5,23,43,23,28,23,0,39,148,...,7,12,5,263085170.0,898,1.0,0.28,0.285714,0.294118,292967.895323


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 292967.895323

# use NAAF >50% ALC to get NAAF factor
peaks126['NAAF factor'] = (peaks126['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_126 = peaks126[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_126['A-NAAF50'] = peaksAAPTM_126['A'] * peaks126['NAAF factor']
peaksAAPTM_126['C-NAAF50'] = peaksAAPTM_126['C'] * peaks126['NAAF factor']
peaksAAPTM_126['D-NAAF50'] = peaksAAPTM_126['D'] * peaks126['NAAF factor']
peaksAAPTM_126['E-NAAF50'] = peaksAAPTM_126['E'] * peaks126['NAAF factor']
peaksAAPTM_126['F-NAAF50'] = peaksAAPTM_126['F'] * peaks126['NAAF factor']
peaksAAPTM_126['G-NAAF50'] = peaksAAPTM_126['G'] * peaks126['NAAF factor']
peaksAAPTM_126['H-NAAF50'] = peaksAAPTM_126['H'] * peaks126['NAAF factor']
peaksAAPTM_126['I-NAAF50'] = peaksAAPTM_126['I'] * peaks126['NAAF factor']
peaksAAPTM_126['K-NAAF50'] = peaksAAPTM_126['K'] * peaks126['NAAF factor']
peaksAAPTM_126['L-NAAF50'] = peaksAAPTM_126['L'] * peaks126['NAAF factor']
peaksAAPTM_126['M-NAAF50'] = peaksAAPTM_126['M'] * peaks126['NAAF factor']
peaksAAPTM_126['N-NAAF50'] = peaksAAPTM_126['N'] * peaks126['NAAF factor']
peaksAAPTM_126['P-NAAF50'] = peaksAAPTM_126['P'] * peaks126['NAAF factor']
peaksAAPTM_126['Q-NAAF50'] = peaksAAPTM_126['Q'] * peaks126['NAAF factor']
peaksAAPTM_126['R-NAAF50'] = peaksAAPTM_126['R'] * peaks126['NAAF factor']
peaksAAPTM_126['S-NAAF50'] = peaksAAPTM_126['S'] * peaks126['NAAF factor']
peaksAAPTM_126['T-NAAF50'] = peaksAAPTM_126['T'] * peaks126['NAAF factor']
peaksAAPTM_126['V-NAAF50'] = peaksAAPTM_126['V'] * peaks126['NAAF factor']
peaksAAPTM_126['W-NAAF50'] = peaksAAPTM_126['W'] * peaks126['NAAF factor']
peaksAAPTM_126['Y-NAAF50'] = peaksAAPTM_126['Y'] * peaks126['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_126['ccarb-NAAF50'] = peaksAAPTM_126['c-carb'] * peaksAAPTM_126['NAAF factor']
peaksAAPTM_126['moxid-NAAF50'] = peaksAAPTM_126['m-oxid'] * peaksAAPTM_126['NAAF factor']
peaksAAPTM_126['ndeam-NAAF50'] = peaksAAPTM_126['n-deam'] * peaksAAPTM_126['NAAF factor']
peaksAAPTM_126['qdeam-NAAF50'] = peaksAAPTM_126['q-deam'] * peaksAAPTM_126['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_126.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50_naaf.csv")

peaksAAPTM_126.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,21.777130,1,0,0,0,0,0,0,0,...,21.777130,65.331391,21.777130,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
1,LPQVEGTGGNVQPSKDLVR,NaN,0,0,1,1,0,3,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VTNLAEVLAR,0.484695,3,0,0,1,0,0,0,0,...,0.484695,0.000000,0.484695,0.969389,0.0,0.0,0.0,0.0,0.484695,0.484695
3,VTNLAEVLAR,NaN,3,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LSSPATLDSR,0.074752,1,0,1,0,0,0,0,0,...,0.074752,0.224257,0.074752,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_126['NAAF factor'].sum(),
        'A': peaksAAPTM_126['A-NAAF50'].sum(),
        'C': peaksAAPTM_126['C-NAAF50'].sum(),
        'D': peaksAAPTM_126['D-NAAF50'].sum(),
        'E': peaksAAPTM_126['E-NAAF50'].sum(),
        'F': peaksAAPTM_126['F-NAAF50'].sum(),
        'G': peaksAAPTM_126['G-NAAF50'].sum(),
        'H': peaksAAPTM_126['H-NAAF50'].sum(),
        'I': peaksAAPTM_126['I-NAAF50'].sum(),
        'K': peaksAAPTM_126['K-NAAF50'].sum(),
        'L': peaksAAPTM_126['L-NAAF50'].sum(),
        'M': peaksAAPTM_126['M-NAAF50'].sum(),
        'N': peaksAAPTM_126['N-NAAF50'].sum(),
        'P': peaksAAPTM_126['P-NAAF50'].sum(),
        'Q': peaksAAPTM_126['Q-NAAF50'].sum(),
        'R': peaksAAPTM_126['R-NAAF50'].sum(),
        'S': peaksAAPTM_126['S-NAAF50'].sum(),
        'T': peaksAAPTM_126['T-NAAF50'].sum(),
        'V': peaksAAPTM_126['V-NAAF50'].sum(),
        'W': peaksAAPTM_126['W-NAAF50'].sum(),
        'Y': peaksAAPTM_126['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_126['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_126['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_126['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_126['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 292967.895323

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,104.510758,98.425316,0.520941,3.169718,6.637625,2.964737,5.402267,1.349357,0.0,4.324658,...,2.896814,0.520941,0.233287,0.650033,0.557283,1.0,0.080751,0.026348,0.410871,0.000357


## Export stripped peptides >50% ALC

In [9]:
##### keep only stripped peptide column 
pep50 = peaks126[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/126_MN_T00_GF_DN/126B_MN_T00_GF_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks126['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks126[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 90
# nonredundant stripped Peaks peptides >50% ALC 72
average peptide length Peaks peptides >50% ALC 9.977777777777778
# redundant Peaks peptides >50% ALC 90
# nonredundant Peaks peptides 72


,stripped_peptide
0,LSSPATLNSR
1,LPQVEGTGGNVQPSKDLVR
2,VTNLAEVLAR
3,VTNLAEVLAR
4,LSSPATLDSR
